https://devblogs.nvidia.com/deep-learning-self-driving-cars/

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [76]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import pickle
import sklearn
import cv2

tf.python.control_flow_ops = tf

from keras.models import Sequential
from keras.layers import Flatten, Dense

In [3]:
DATA_DIR = '../bclone-data-standard'

In [4]:
import bclone

In [86]:
def open_images(data_dir, log_df, controls):
    
    cameras = ('center', 'left', 'right')
    
    im_list = []
    controls_list = []
    for i in range(len(log_df)):
        line = log_df.iloc[i]
        controls_vals = line[controls]
        for c in cameras:
            
            imfile = os.path.join(data_dir, line[c].strip())
            im = cv2.imread(imfile)

            im_list.append(im)
            controls_list.append(controls_vals)
            
    X = np.array(im_list)
    y = np.array(controls_list)
    
    if y.shape[1] == 1:
        y = y.reshape(-1)
            
    return X, y
        
        
def data_generator(data_dir, batch_size=32, controls=['steering', 'throttle', 'speed', 'brake']):
    
    log_df = sklearn.utils.shuffle( bclone.load_log(data_dir) )
    n = len(controls)
    
    for offset in range(0, n, batch_size):
        log_subset = log_df[offset:offset+batch_size]
        X_batch, y_batch = open_images(data_dir, log_subset) 
        yield sklearn.utils.shuffle(X_batch, y_batch)

In [27]:
log_df = bclone.load_log(DATA_DIR)

In [90]:
#a, b = open_images(DATA_DIR, log_df[:5], ['steering'])

In [106]:
model = Sequential()
model.add(Flatten(input_shape=(160, 320, 3)))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

[How to Use Generators](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/46a70500-493e-4057-a78e-b3075933709d/concepts/b602658e-8a68-44e5-9f0b-dfa746a0cc1a)

In [108]:
#history = model.fit_generator(train_generator, samples_per_epoch= /
#            len(train_samples), validation_data=validation_generator, /
#            nb_val_samples=len(validation_samples), nb_epoch=3)